# Training and A/B test for Conversion agents - <font color='blue'> Presence</font> of sale

In [16]:
import gym, recogym
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import pandas as pd
import pickle as pkl
pd.options.mode.chained_assignment = None 
# from inspect import getsource
from recogym.evaluate_agent_sale import verify_agents_sale, display_metrics, verify_agents_sale_extended 
from tqdm import tqdm

# env_1_sale_args is a dictionary of default parameters (i.e. number of products)
from recogym import env_1_sale_args, Configuration
from recogym.agents.sale_agent import train_agents, train_timeagents
from recogym.envs.utils_sale import format_avg_result, avg_result, format_avg_result_extended, avg_result_extended


# You can overwrite environment arguments here:
env_1_sale_args['random_seed'] = 0
env_1_sale_args['num_products'] = 10
env_1_sale_args['number_of_flips'] = 10 

num_products = env_1_sale_args['num_products']
print('Number of products =',num_products)
print('Number of flips =',env_1_sale_args['number_of_flips'])
nb_flips = env_1_sale_args['number_of_flips']

# You can overwrite environment arguments here:
env_1_sale_args['random_seed'] = 42
env_1_sale_args['mu_sale'] = False 

# env_1_sale_args['kappa'] = 0.5
print('Value of kappa =',env_1_sale_args['kappa'])

# Initialize the gym 
env = gym.make('reco-gym-sale-v1')
env.init_gym(env_1_sale_args)



Number of products = 10
Number of flips = 10
Value of kappa = 0.2


In [17]:
# Repository to save pickles
data_repo = 'data_conversion/'

## Train baseline agents

#### Settings

In [18]:
## Choose number of users for training and AB test
# Number of users for the training
# env_1_sale_args['num_users'] = 6 ##tochange !!
env_1_sale_args['num_users'] = 5000
num_users = env_1_sale_args['num_users']

# Number of users for the A/B test
# env_1_sale_args['num_users_AB'] = 7 ##tochange !!
env_1_sale_args['num_users_AB'] = 5000
num_users_AB = env_1_sale_args['num_users_AB']

# Choose user features
from recogym.agents.sale_agent import CountViewsClicksFeatureProvider, CountViewsFeatureProvider, ShareViewsClicksFeatureProvider, ShareViewsFeatureProvider
vc_feature = CountViewsClicksFeatureProvider(env.config)
v_feature = CountViewsFeatureProvider(env.config)
vc_share_feature = ShareViewsClicksFeatureProvider(env.config)
v_share_feature = ShareViewsFeatureProvider(env.config)
features = {'vc':vc_feature,
           'v':v_feature,
           'vc_share':vc_share_feature,
           'v_share':v_share_feature}
feature_name = 'v_share'
feature = features[feature_name]

#### <font color='red'> Number of A/B tests</font>

In [19]:
# # Choose number of A/B tests
# num_AB_tests = 2 ##tochange !
num_AB_tests = 25 ##tochange !

#### Logs

In [20]:
agents={}
logs={}

############## Random agent
name_agent = 'rand'+str(nb_flips)
from recogym.agents import RandomAgent, random_args
random_agent = RandomAgent(Configuration(random_args))
agents[name_agent] = random_agent


try:
    logs[name_agent] = pkl.load(open(data_repo + 'data' + str(num_users) + name_agent + '.pkl','rb'))
    print('--- Logs loaded---')
except: 
    print("--- Generate logs ---")
    logs[name_agent] = deepcopy(env).generate_logs(num_users)
    print(data_repo + 'data' + str(num_users) + name_agent + '.pkl')
    pkl.dump(logs[name_agent], open(data_repo + 'data' + str(num_users) + name_agent + '.pkl','wb'))

--- Logs loaded---


#### Training for the <font color='blue'> Presence</font> of sale

##### No weights

In [21]:
name_extension = 'pres'
name_logging = name_agent
res_dict = {}
try :
    info, save_agents = pkl.load(open(data_repo+str('agents'+str(num_users)+name_logging+feature_name+name_extension+'.pkl'),'rb'))
    print('--- Trained agents loaded --- ')
except :
    print('--- Train agents ---')
    info, save_agents = train_agents(name_logging,logs,feature_name,features, num_users=num_users, 
                                     kronecker_features=True,additional_models_only=False, repo = data_repo)

--- Trained agents loaded --- 


In [22]:
try:
    for i in range(num_AB_tests):
        name_extension = 'pres'+'_nb'+str(i)
        name = 'likelihood_saleclickprod_discount_allpres_full'
        agent_id = name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension
        print(str(data_repo) +'res_'+agent_id+name+'.pkl')
        r = pkl.load(open(str(data_repo) +'res_'+agent_id+name+'.pkl',"rb"))
        for k in r.keys():
            if k not in ['CTR', 'Tot sales ATT', 'Share user with sale ATT', 'Tot sales', 'Share user with sale',
                         'True CTR','True PCS','True OS','True NCS']:
                r[k] = None

        res_dict[name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension] = r
    print('--- A/B test results loaded ---')
    
except:
    print(' --- Run A/B test --- ')
    for i in range(num_AB_tests):
        print("-------------- A/B test n°"+str(i)+'--------------')
        name_extension = 'pres'+'_nb'+str(i)
        res=verify_agents_sale_extended(
            env,
            number_of_users=num_users_AB,
            agents={
                **agents, 
                **save_agents},
            name = name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension,
            seed = 0,
            same_env = False,
            repo = data_repo
        )

        r = deepcopy(res)
        for k in r.keys():
            if k not in ['CTR', 'Tot sales ATT', 'Share user with sale ATT', 'Tot sales', 'Share user with sale',
                         'True CTR','True PCS','True OS','True NCS']:
                r[k] = None

        res_dict[name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension] = r

data_conversion/res_rand106_7_v_sharepres_nb0likelihood_saleclickprod_discount_allpres_full.pkl
data_conversion/res_rand106_7_v_sharepres_nb1likelihood_saleclickprod_discount_allpres_full.pkl
--- A/B test results loaded ---


In [23]:
res_dict

{'rand106_7_v_sharepres_nb0': {'CTR':                                        Agent     0.025     0.500     0.975
  0                                     rand10  0.005552  0.014235  0.029072
  1                likelihood_logreg_clickpres  0.007335  0.015389  0.027806
  2                likelihood_logreg_postdpres  0.004092  0.008233  0.014503
  3           likelihood_saleclickprod_allpres  0.006319  0.014910  0.028986
  4  likelihood_saleclickprod_discount_allpres  0.012240  0.022909  0.038338,
  'Tot sales ATT':                                        Agent  TotSalesAtt
  0                                     rand10            1
  1                likelihood_logreg_clickpres            3
  2                likelihood_logreg_postdpres            1
  3           likelihood_saleclickprod_allpres            3
  4  likelihood_saleclickprod_discount_allpres            4,
  'Share user with sale ATT':                                        Agent     0.025     0.500     0.975
  0               

In [24]:
res_avg = avg_result_extended(res_dict)
(res_recap, res_recap_latex, 
 res_AB, res_AB_latex, 
 res_true, res_true_latex) = format_avg_result_extended(res_avg) #get dataframe & corresponding latex table

-- A/B test --


,Agent,CTR,Att Sales,Att CR,Sales,CR
0,Rand,0.658 (0.076) %,0 (0),10.06 (0.00) %,6 (1),33.97 (8.45) %
1,Click,0.680 (0.126) %,1 (0),16.03 (8.45) %,5 (0),22.01 (8.46) %
2,PDS,0.406 (0.008) %,0 (0),10.06 (0.00) %,4 (4),25.00 (12.69) %
3,PCS,0.995 (0.353) %,1 (0),22.01 (0.00) %,5 (1),27.99 (0.00) %
4,DPCS,0.839 (0.434) %,1 (0),16.03 (0.00) %,7 (0),33.97 (8.45) %


-- True metrics --


,Agent,True CTR,True PCS,True OS,True NCS,NDPC,DPCSO,DPCSN
0,Rand,0.548 (0.024) %,4.60 (0.05) %,3.63 (0.33) %,3.60 (0.28) %,0.504 (0.016) pm,0.107 (0.035) pm,0.110 (0.029) pm
1,Click,0.499 (0.023) %,6.07 (0.12) %,4.62 (0.89) %,5.28 (0.22) %,0.606 (0.040) pm,0.143 (0.071) pm,0.079 (0.006) pm
2,PDS,0.514 (0.019) %,4.11 (0.08) %,2.97 (0.11) %,2.83 (0.08) %,0.423 (0.024) pm,0.117 (0.001) pm,0.132 (0.021) pm
3,PCS,0.740 (0.010) %,5.36 (0.35) %,3.86 (1.21) %,4.47 (0.46) %,0.793 (0.041) pm,0.224 (0.130) pm,0.132 (0.018) pm
4,DPCS,0.738 (0.234) %,6.60 (0.44) %,5.27 (0.54) %,6.18 (0.76) %,0.985 (0.374) pm,0.195 (0.048) pm,0.056 (0.027) pm


In [25]:
print(res_AB_latex+'\n')
print(res_true_latex)

\begin{tabular}{llllll}
\toprule
 Agent &              CTR & Att Sales &          Att CR &  Sales &               CR \\
\midrule
  Rand &  0.658 (0.076) \% &     0 (0) &  10.06 (0.00) \% &  6 (1) &   33.97 (8.45) \% \\
 Click &  0.680 (0.126) \% &     1 (0) &  16.03 (8.45) \% &  5 (0) &   22.01 (8.46) \% \\
   PDS &  0.406 (0.008) \% &     0 (0) &  10.06 (0.00) \% &  4 (4) &  25.00 (12.69) \% \\
   PCS &  0.995 (0.353) \% &     1 (0) &  22.01 (0.00) \% &  5 (1) &   27.99 (0.00) \% \\
  DPCS &  0.839 (0.434) \% &     1 (0) &  16.03 (0.00) \% &  7 (0) &   33.97 (8.45) \% \\
\bottomrule
\end{tabular}


\begin{tabular}{llllllll}
\toprule
 Agent &         True CTR &       True PCS &        True OS &       True NCS &              NDPC &             DPCSO &             DPCSN \\
\midrule
  Rand &  0.548 (0.024) \% &  4.60 (0.05) \% &  3.63 (0.33) \% &  3.60 (0.28) \% &  0.504 (0.016) pm &  0.107 (0.035) pm &  0.110 (0.029) pm \\
 Click &  0.499 (0.023) \% &  6.07 (0.12) \% &  4.62 (0.89) \% & 

##### Weights

In [26]:
name_extension = 'presweights'
name_logging = name_agent
res_dict = {}
try :
    info, save_agents = pkl.load(open(data_repo+str('agents'+str(num_users)+name_logging+feature_name+name_extension+'.pkl'),'rb'))
    print('--- Trained agents loaded --- ')
except :
    print('--- Train agents ---')
    info, save_agents = train_agents(name_logging,logs,feature_name,features, num_users=num_users, 
                                     kronecker_features=True,weights = True, additional_models_only=False, repo = data_repo)

--- Trained agents loaded --- 


In [27]:
try:
    for i in range(num_AB_tests):
        name_extension = 'presweights'+'_nb'+str(i)
        name = 'likelihood_saleclickprod_discount_allpresweights_full'
        agent_id = name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension
        r = pkl.load(open(str(data_repo) +'res_'+agent_id+name+'.pkl',"rb"))
        for k in r.keys():
            if k not in ['CTR', 'Tot sales ATT', 'Share user with sale ATT', 'Tot sales', 'Share user with sale',
                         'True CTR','True PCS','True OS','True NCS']:
                r[k] = None

        res_dict[name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension] = r
    print('--- A/B test results loaded ---')
    
except:
    print(' --- Run A/B test --- ')
    for i in range(num_AB_tests):
        print("-------------- A/B test n°"+str(i)+'--------------')
        name_extension = 'presweights'+'_nb'+str(i)
        res=verify_agents_sale_extended(
            env,
            number_of_users=num_users_AB,
            agents={
                **agents, 
                **save_agents},
            name = name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension,
            seed = 0,
            same_env = False,
            repo = data_repo
        )

        r = deepcopy(res)
        for k in r.keys():
            if k not in ['CTR', 'Tot sales ATT', 'Share user with sale ATT', 'Tot sales', 'Share user with sale',
                         'True CTR','True PCS','True OS','True NCS']:
                r[k] = None

        res_dict[name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension] = r

--- A/B test results loaded ---


In [28]:
res_avg = avg_result_extended(res_dict)
(res_recap, res_recap_latex, 
 res_AB, res_AB_latex, 
 res_true, res_true_latex) = format_avg_result_extended(res_avg) #get dataframe & corresponding latex table

-- A/B test --


,Agent,CTR,Att Sales,Att CR,Sales,CR
0,Rand,0.706 (0.060) %,1 (0),13.04 (4.22) %,8 (7),22.01 (8.46) %
1,Click,0.659 (0.067) %,1 (0),16.03 (0.00) %,4 (1),25.00 (4.23) %
2,PDS,0.950 (0.238) %,2 (0),25.00 (4.23) %,8 (4),33.97 (8.45) %
3,PCS,0.609 (0.300) %,2 (1),22.01 (8.46) %,11 (4),36.96 (4.22) %
4,DPCS,0.823 (0.522) %,0 (0),4.15 (0.00) %,1 (1),16.03 (8.45) %


-- True metrics --


,Agent,True CTR,True PCS,True OS,True NCS,NDPC,DPCSO,DPCSN
0,Rand,0.611 (0.012) %,5.39 (0.30) %,3.91 (0.59) %,4.62 (0.33) %,0.659 (0.049) pm,0.180 (0.032) pm,0.094 (0.002) pm
1,Click,0.653 (0.179) %,4.93 (0.24) %,3.35 (0.61) %,3.80 (0.64) %,0.647 (0.207) pm,0.199 (0.008) pm,0.139 (0.012) pm
2,PDS,0.623 (0.107) %,5.21 (1.10) %,4.38 (1.65) %,4.66 (1.33) %,0.638 (0.026) pm,0.110 (0.086) pm,0.071 (0.040) pm
3,PCS,0.490 (0.125) %,6.15 (0.81) %,5.20 (1.03) %,5.54 (1.05) %,0.592 (0.074) pm,0.095 (0.045) pm,0.062 (0.038) pm
4,DPCS,0.858 (0.012) %,4.12 (1.32) %,2.11 (0.91) %,3.04 (1.53) %,0.705 (0.217) pm,0.345 (0.066) pm,0.186 (0.039) pm


In [29]:
print(res_AB_latex+'\n')
print(res_true_latex)

\begin{tabular}{llllll}
\toprule
 Agent &              CTR & Att Sales &          Att CR &   Sales &              CR \\
\midrule
  Rand &  0.706 (0.060) \% &     1 (0) &  13.04 (4.22) \% &   8 (7) &  22.01 (8.46) \% \\
 Click &  0.659 (0.067) \% &     1 (0) &  16.03 (0.00) \% &   4 (1) &  25.00 (4.23) \% \\
   PDS &  0.950 (0.238) \% &     2 (0) &  25.00 (4.23) \% &   8 (4) &  33.97 (8.45) \% \\
   PCS &  0.609 (0.300) \% &     2 (1) &  22.01 (8.46) \% &  11 (4) &  36.96 (4.22) \% \\
  DPCS &  0.823 (0.522) \% &     0 (0) &   4.15 (0.00) \% &   1 (1) &  16.03 (8.45) \% \\
\bottomrule
\end{tabular}


\begin{tabular}{llllllll}
\toprule
 Agent &         True CTR &       True PCS &        True OS &       True NCS &              NDPC &             DPCSO &             DPCSN \\
\midrule
  Rand &  0.611 (0.012) \% &  5.39 (0.30) \% &  3.91 (0.59) \% &  4.62 (0.33) \% &  0.659 (0.049) pm &  0.180 (0.032) pm &  0.094 (0.002) pm \\
 Click &  0.653 (0.179) \% &  4.93 (0.24) \% &  3.35 (0.61) \% & 